# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import json
import csv
import numpy as np
import pandas as pd
import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core.conda_dependencies import CondaDependencies

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive-run'

experiment=Experiment(ws, experiment_name)

In [ ]:
# Load the registered dataset from workspace
dataset = Dataset.get_by_name(ws, name='diabetes')

# Convert the dataset to dataframe
df = dataset.to_pandas_dataframe()
df.info()

In [ ]:
amlcompute_cluster_name = "project-cluster"

# Verify if cluster does not exist otherwise use the existing one
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_V2',
                                                           vm_priority = 'lowpriority', 
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

In [ ]:
# create environment
environment = Environment(name="azure-env")
conda_env = CondaDependencies()

# Needed packages and scripts
conda_env.add_conda_package("pandas")
conda_env.add_conda_package("numpy")
conda_env.add_conda_package("scikit-learn")
conda_env.add_conda_package("sklearn")
conda_env.add_conda_package("impyute")
conda_env.add_pip_package("azureml-defaults")

# Adding dependencies to the created environment
environment.python.conda_dependencies=conda_env

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# Create an early termination policy
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

# Create different params that will be used during training
param_sampling = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1.0,10.0,50.0,100,1000),
        '--max_iter': choice(10,50)
    }
)

# Create estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory = ".",
                            script='train.py',
                            arguments=['--data', dataset.id],
                            compute_target=amlcompute_cluster_name,
                            environment=environment)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [ ]:
# Submit experiment
run = experiment.submit(hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(run).show()

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
run

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Retrieve and save best model
best_run = run.get_best_run_by_primary_metric()
best_run.get_file_names()

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Registring the best model
model = best_run.register_model(model_name='hd-best-model', model_path='outputs/model.joblib')

In [ ]:
inference_config = InferenceConfig(entry_script='score.py',
                                   environment=environment)
service_name = 'hd-deploy-1'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True
                      )
service.wait_for_deployment(show_output=True)

In [ ]:
# Enable app insights
service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
data = [{"Pregnancies": 6, 
 "Glucose": 148, 
 "BloodPressure": 72, 
 "SkinThickness": 35, 
 "Insulin": 0, 
 "BMI": 33.5, 
 "DiabetesPedigreeFunction": 0.627, 
 "Age": 50},
        
{"Pregnancies": 1, 
 "Glucose": 85, 
 "BloodPressure": 66, 
 "SkinThickness": 29, 
 "Insulin": 20, 
 "BMI": 26.5, 
 "DiabetesPedigreeFunction": 0.351, 
 "Age": 31}]

print(data)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
import json


input_payload = json.dumps({
    'data': df[0:2].values.tolist(),
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

In [ ]:
logs = service.get_logs()
logs

In [ ]:
service.delete()